# Introduction
State notebook purpose here

## Standard imports

In [1]:
# Data manipulation
import pandas as pd
import numpy as np

# Options for pandas
pd.options.display.max_columns = 50
pd.options.display.max_rows = 30

from IPython import get_ipython
ipython = get_ipython()

# autoreload extension
if 'autoreload' not in ipython.extension_manager.loaded:
    %load_ext autoreload

%autoreload 2

import matplotlib.pyplot as plt
from matplotlib import gridspec
%matplotlib inline

import time
np.random.seed(int(time.time()))

## Specific imports

In [2]:
from generate_timeseries import *

# Analysis/Modeling
Do work here

In [9]:
N = 500

params = {}

steadystate = np.logspace(-3, 2, N).reshape([N, 1])

# no interaction
omega = np.zeros([N, N]);
np.fill_diagonal(omega, -1)

params['interaction_matrix'] = omega

# no immigration
params['immigration_rate'] = np.zeros([N, 1])

# different growth rates determined by the steady state
params['growth_rate'] = - (omega).dot(steadystate)

params['initial_condition'] = np.copy(steadystate) * np.random.normal(1, 0.1, steadystate.shape)

params['noise_linear'] = 1e-1

params['maximum_capacity'] = 1

%load_ext line_profiler

%lprun -f Timeseries2.add_step Timeseries2(params)

#%timeit Timeseries(params, noise_implementation=NOISE.LANGEVIN_LINEAR, model = MODEL.MAX, dt=0.01, tskip=49, T=200.0, seed=int(time.time()))

#%timeit Timeseries_old(params, noise_implementation=NOISE.LANGEVIN_LINEAR, model = MODEL.MAX, dt=0.01, tskip=49, T=200.0, seed=int(time.time()))

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


# Results
Show graphs and stats here

# Conclusions and Next Steps
Summarize findings here

In [6]:
class Timeseries2():
    def __init__(self, params, model=MODEL.GLV, noise_implementation=NOISE.LANGEVIN_LINEAR, dt=0.01, T=100, tskip=0,
                 f=0, seed=None):
        self.params = params
        self.model = model
        self.noise_implementation = noise_implementation
        self.dt = dt
        self.T = T
        self.tskip = tskip
        self.f = f
        self.seed = seed

        self.set_seed()

        self.check_input_parameters()
        self.set_parameters(params)

        self.init_Nspecies_Nmetabolites()

        #self.deterministic_step = self.deterministic_step_function()
        #self.stochastic_step = self.stochastic_step_function()
        #self.add_step = self.add_step_function()

        if self.model in [MODEL.GLV, MODEL.MAX]:
            self.x = np.copy(self.params['initial_condition'])
        elif self.model == MODEL.QSMI:
            self.x = np.copy(self.params['initial_condition'])[:len(self.params['d'])]  # initial state species
            self.y = np.copy(self.params['initial_condition'])[len(self.params['d']):]  # initial state metabolites ad
        self.x_ts = np.copy(self.x)
        
        if self.model in [MODEL.GLV, MODEL.MAX]:
            self.interaction_matrix_pos = self.interaction_matrix.copy(); 
            self.interaction_matrix_pos[self.interaction_matrix < 0] = 0;
            
            self.interaction_matrix_neg = self.interaction_matrix.copy(); 
            self.interaction_matrix_neg[self.interaction_matrix > 0] = 0;
            
            self.growth_rate_pos = self.growth_rate.copy(); 
            self.growth_rate_pos[self.growth_rate < 0] = 0;
            
            self.growth_rate_neg = self.growth_rate.copy(); 
            self.growth_rate_neg[self.growth_rate > 0] = 0;

        if f != 0:
            self.write_header()

        self.integrate()

    def set_seed(self):
        if self.seed == None:
            np.random.seed(int(time.time()))
        else:
            np.random.seed(self.seed)

    def set_parameters(self, params):
        for key, value in params.items():
            setattr(self, key, value)

    def check_input_parameters(self):
        # Function to check if all necessary parameters where provided, raises error if parameters are missing

        if self.model in [MODEL.GLV, MODEL.MAX]:
            parlist = ['interaction_matrix', 'immigration_rate', 'growth_rate', 'initial_condition']

            if 'LINEAR' in self.noise_implementation.name:
                parlist += ['noise_linear']
            elif 'SQRT' in self.noise_implementation.name:
                parlist += ['noise_sqrt']
            elif 'CONSTANT' in self.noise_implementation.name:
                parlist += ['noise_constant']
            elif 'INTERACTION' in self.noise_implementation.name:
                parlist += ['noise_interaction']

            if self.model in [MODEL.MAX]:
                parlist += ['maximum_capacity']

        elif self.model == MODEL.QSMI:
            parlist = ['psi', 'd', 'g', 'dm', 'kappa', 'phi', 'initcond', 'noise']

        if 'MAX' in self.noise_implementation.name:
            parlist += ['maximum_capacity']

        for par in parlist:
            if not par in self.params:
                raise KeyError('Parameter %s needs to be specified for the %s model and %s noise implementation.' % (
                    par, self.model.name, self.noise_implementation.name))

        # check whether matrix shapes are correct
        if self.model == MODEL.GLV:
            if not np.all(len(row) == len(self.params['interaction_matrix']) for row in
                          self.params['interaction_matrix']):
                raise ValueError('Interaction matrix is not square.')

            for parname in ['immigration_rate', 'growth_rate', 'initial_condition']:
                if np.any(self.params[parname].shape != (self.params['interaction_matrix'].shape[0], 1)):
                    raise ValueError('%s has the incorrect shape: %s instead of (%d,1)' % (
                        parname, str(self.params[parname].shape), self.params['interaction_matrix'].shape[0]))

    def write_header(self):
        # Write down header in file f
        with open(self.f, "a") as file:
            file.write("time")
            for k in range(1, self.Nspecies + 1):
                file.write(",species_%d" % k)
            for k in range(1, self.Nmetabolites + 1):
                file.write(",metabolite_%d" % k)

            file.write("\n")

            file.write("%.3E" % 0)
            for k in self.initial_condition:
                file.write(",%.3E" % k)
            file.write("\n")

    def init_Nspecies_Nmetabolites(self):
        if self.model in [MODEL.GLV, MODEL.MAX]:
            self.Nspecies = len(self.interaction_matrix)  # number of species
            self.Nmetabolites = 0  # number of metabolites, 0 in the GLV models
        elif self.model == MODEL.QSMI:
            self.Nspecies = len(self.d)  # number of species
            self.Nmetabolites = len(self.dm)  # number of metabolites

    def integrate(self):
        # If noise is Ito, first generate brownian motion.
        if self.noise_implementation == NOISE.ARATO_LINEAR:
            self.xt = np.zeros_like(self.initial_condition)
            self.bm = brownian(np.zeros(len(self.initial_condition)), int(self.T / self.dt), self.dt, 1,
                               out=None)

        x_ts = np.zeros([int(self.T / (self.dt * (self.tskip + 1))), self.Nspecies])

        # set initial condition
        x_ts[0] = self.x.flatten()
        
        i = 0
        
        # Integrate ODEs according to model and noise
        while i < int(self.T / (self.dt * (self.tskip + 1))) and not np.all(np.isnan(self.x)):
            for j in range(self.tskip + 1):
                self.add_step(i * (self.tskip + 1) + j)

            # Save abundances
            if self.f != 0:
                self.write_abundances_to_file(i * (self.tskip + 1) + j)

            x_ts[i] = self.x.flatten()

            i += 1

        # dataframe to save timeseries
        self.x_ts = pd.DataFrame(x_ts, columns=['species_%d' % i for i in range(1, self.Nspecies + 1)])
        self.x_ts['time'] = (self.dt * (self.tskip + 1) * np.arange(0, int(self.T / (self.dt * (self.tskip + 1)))))

        return

    def add_step(self, i):
        prob_to_grow = self.probability_to_grow()
        dx_det = self.deterministic_step(prob_to_grow) 
        dx_stoch = self.stochastic_step(prob_to_grow)

        self.x += dx_det + dx_stoch

        # abundance cannot be negative
        self.x[self.x < 0] = 0

    def probability_to_grow(self):
        return 1 - max(0, min(1, np.sum(self.x)/self.maximum_capacity))
        
    def deterministic_step(self, prob_to_growth):
        return ( self.immigration_rate * prob_to_growth + (
            (self.interaction_matrix_pos * prob_to_growth + self.interaction_matrix_neg).dot(self.x) 
            + self.growth_rate_pos * prob_to_growth + self.growth_rate_neg ) * self.x) * self.dt
       
    def stochastic_step(self, prob_to_growth):
        g = np.random.standard_normal(self.x.shape)
        g[g > 0] *= prob_to_growth
        return self.noise_linear * self.x * np.sqrt(self.dt) * g

    def ricker_step(self):
        if self.noise_implementation == NOISE.RICKER_LINEAR:
            if self.noise_linear == 0:
                b = np.ones(self.x.shape)
            else:
                b = np.exp(self.noise_linear * np.sqrt(self.dt) * np.random.standard_normal(self.x.shape))
            self.x = b * self.x * np.exp(self.interaction_matrix.dot(
                self.x + np.linalg.inv(self.interaction_matrix).dot(self.growth_rate)) * self.dt)
        else:
            raise ValueError('No implementation for "%s"' % self.noise_implementation.name)

    def arato_step(self, i):
        if self.noise_implementation == NOISE.ARATO_LINEAR:
            self.xt += self.x * self.dt

            t = i * self.dt

            Y = self.growth_rate * t - self.noise_linear ** 2 / 2 * t + self.interaction_matrix.dot(self.xt) + \
                self.noise_linear * self.bm[:, i].reshape(self.x.shape)  # noise * np.random.normal(0, 1, initcond.shape)
            self.x = self.initial_condition * np.exp(Y)

    def write_abundances_to_file(self, i):
        with open(self.f, "a") as file:
            file.write("%.5E" % (i * self.dt))
            for k in self.x:
                file.write(",%.5E" % k)
            if self.model == MODEL.QSMI:
                for k in self.y:
                    file.write(",%.5E" % k)
            file.write("\n")

    @property
    def timeseries(self):
        #make sure indices in right order

        columns = ['time'] + ['species_%d' % i for i in range(1, self.Nspecies + 1)]
        self.x_ts = self.x_ts[columns]

        return self.x_ts

    @property
    def endpoint(self):
        df = pd.DataFrame(self.x, columns=['endpoint'], index=['species_%d' % i for i in range(1, self.Nspecies+1)])
        return df
